In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

In [2]:
df = pd.read_excel('data/fy-2075-76.xlsx', header=1)

In [3]:
def subheading_fix(x):
    if pd.notna(x) and type(x) == datetime and " " in str(x):
        return f'{x.year}.{x.month}.{x.day}'
    return x
df = df[df.iloc[:, 0] != "Heading"]
df = df[df.iloc[:, 4] != 'SAARC']
df = df[df.iloc[:, 0] != 1]
df.columns = ["Heading","Sub-heading","Description of goods","Import Duty SAARC", "Import Duty General"]
df['Sub-heading'] = df['Sub-heading'].apply(subheading_fix)
df["Description of goods"] = df["Description of goods"].apply(lambda x: " ".join(x.split()))
df.reset_index(drop=True, inplace=True)

In [4]:
df['Heading'] = np.NaN
main_descriptors = df[~df['Description of goods'].str.startswith('-')].index
for i, ind in enumerate(main_descriptors):
    df.loc[ind, 'Heading'] = f'87.{"0" if i<9 else ""}{i+1}'

/tmp/ipykernel_25088/1324436002.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '87.01' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[ind, 'Heading'] = f'87.{"0" if i<9 else ""}{i+1}'


In [5]:
def count_hyphens_in_prefix(s):
    count = 0
    for char in s:
        if char == '-':
            count += 1
        else:
            break  # Exit the loop if a non-hyphen character is encountered
    return count

for i, row in df[::-1].iterrows():
    if not pd.notna(row['Sub-heading']):
        if pd.notna(row['Heading']):
            df.loc[i, 'Sub-heading'] = row['Heading'].replace('.', '')
        else:
            dashes = count_hyphens_in_prefix(row['Description of goods'])
            prev_code = df.loc[i+1]['Sub-heading']
            if type(prev_code) == str:
                if prev_code.endswith('.00'):
                    prev_code = prev_code[:len(prev_code)-3]
                if prev_code.endswith('0'):
                    new_code = '.'.join(prev_code.split('.')[:-1])
                else:
                    new_code = prev_code[:len(prev_code)-1]+'0'
                df.loc[i, 'Sub-heading'] = new_code

df['Heading'] = df['Heading'].ffill()

In [6]:
df['Description of goods'] = df['Description of goods'].str.lstrip('-')

def remove_trailing_zeros(x):
    while x.endswith('.00'):
        x = x[:len(x)-3]
    return x

df['Sub-heading'] = df['Sub-heading'].apply(remove_trailing_zeros)

df = df[['Sub-heading', 'Description of goods', 'Import Duty SAARC', 'Import Duty General']]

In [7]:
df.to_csv('data/fy-2075-76.csv', index=False)